In [14]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'businesstimes_com_sg'
site_name = 'The Business Times'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
publish_source = 'businesstimes.com.sg'
country = 'Singapore'
language = 'English'
images_path = []
foot_fall = c.get_foot_fall(publish_source)
cookies = {
    'XywkqJyC': 'AEybcm14AQAAhDupNHa18aOes4nu8JcBdze6C46z2fW9xKhnZNvqK4izCpV6|1|1|496e119641b20ed189a9f600d5e3292e8bd37c75',
    'juLD4H3B': 'ABoOsVh6AQAAXiBvXTtJUVp3Y6OhR4xWj0Wz0FLFIWMr7GFyT-YP1EhgMTfK',
    '_ga': 'GA1.3.429338857.1616743939',
    '_fbp': 'fb.2.1616743940045.1623211001',
    '_hjid': '62234f58-60b4-4798-9c33-90d464a14e32',
    'optimizelyEndUserId': 'oeu1616743940888r0.7228898367865182',
    '_cb_ls': '1',
    '_cb': 'DIt1_8DEtYWoCaF7Ow',
    '_chartbeat2': '.1616743943907.1624985662923.0000000000000001.CkmIYs25JZnDqpn2sD2YilQBHfSwD.1',
    '_ga_M8PMESFDLF': 'GS1.1.1624985663.4.0.1624985663.60',
    '_ncid': '806433bfb1bf32f62baf46a1975f408a',
    '_cc_id': '7ad54f32f5abdebbab1085542f4ec3e7',
    'suid': '31af00bece494193890519164a414d7a',
    '__gads': 'ID=fa51c528a3ff4d00-2283dda7dac60000:T=1616744076:S=ALNI_Mby3aD268CVfJGmf7eOW7fAWS_Ccw',
    'has_js': '1',
    'topOverlayImpressionsServed': '1',
    'NSC_JOlo3vprczwsrc0em1nifnbukr3oebt': 'ffffffff09a3792645525d5f4f58455e445a4a423660',
    '_gid': 'GA1.3.1199293158.1624985652',
    'UserFirstVisit': '1',
    'outbrain_cid_fetch': 'true',
    'X-AMAGENT-TX': 'tyRZy7u/UJSZudeKvZENm+kKp8xVGH0OAJGN05cfhcQ+WsK/9nZywfw9AwR079CPeSajprb7C22IfXmBl8vb94YEvzgOpJV4IC0Y2+5pCptxyOqdGCkobokceJKHj2IBwlWLByHMllJ/BPbayyllXe/gb/N/zxNIqE22gBUbgEgLfaG54OAzIZTqPLtPfY9on70+8xjOJO4+XD2sihXhLA==',
    '_cb_svref': 'null',
    '_dc_gtm_UA-21603044-1': '1',
    '_gcl_au': '1.1.1564330648.1624985670',
    'querylytooltipclose': '1',
    '_hjIncludedInPageviewSample': '1',
    '_hjTLDTest': '1',
    '_hjAbsoluteSessionInProgress': '0',
    '_gat_UA-21603044-1': '1',
    'sessionStatus': '1',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'TE': 'Trailers',
}

home_page= requests.get('https://www.businesstimes.com.sg/', headers=headers, cookies=cookies, timeout=120).text

home_page = c.scrap('<ul\s*class="list-group">(.*?)<div\s*class="col-md-4\s*visible-md\s*visible-lg">', home_page)

for _ in home_page.split('<a href=')[1:-3]:
    
    cat_url = c.scrap('"(.*?)"', _)  
    if 'http' not in cat_url:
        cat_url = 'https://www.businesstimes.com.sg' +  cat_url
    logger.info(f'Fetching cat url  {cat_url}\n')
    cat_page = requests.get(cat_url, headers=headers, cookies=cookies, timeout=120).text
    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) # writes error message with error code
        unable_to_fetch_cat_url += 1
        continue    
    if 'class="col-lg-12' in cat_page:  
        cat_pages=c.scrap('<div\s*class="col-lg-12(.*?)"pager\s*pager-show-more"', cat_page)
    else:
         cat_pages=c.scrap('<div\s*class="col-lg-8(.*?)"pager\s*pager-show-more"', cat_page)
    for i in cat_pages.split('<h2 class="widget__title">')[1:]:
        # source_link
        source_link = c.scrap('<a\s*href="(.*?)"', i)
        
        if 'http' not in source_link:
            source_link = 'https://www.businesstimes.com.sg' +  source_link

        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))

        logger.info(f'Fetching {source_link}\n')
        
        page = requests.get(source_link, headers=headers, cookies=cookies, timeout=120).text 
        
        if page.startswith('Unable to fetch'):     
            logger.info(page) # writes error message with error code
            unable_to_fetch_article_url += 1
            continue    

        source_headline = c.scrap('<title>(.*?)</title>', page)
        
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
            
         # Date and time
        pub_date, publish_time = '', ''

        try:
        
            date_time_str = c.scrap('"datePublished":(.*?)\+', page) 
            if not date_time_str:              
                date_time_str = c.scrap('published_time.*?content="(.*?)\+', page)  
            date_time_str = re.sub('[^\w+]', '', date_time_str)  
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%dT%H%M%S')
            ist_date_time = date_time_obj - timedelta(hours = 2,minutes = 30)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
            
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')            
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
        if pub_date != created_on:
            break    
        # source_content         
        source_content= c.scrap('div\s*class="field\s*field-name-body.*?>(.*?)<section\s*id=',page)    
        if not source_content:
            source_content= c.scrap('<div\s*class="field-body">(.*?)<section\s*id=',page)
        source_content = c.strip_html(source_content)
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')            
            skipped_due_to_content += 1
            continue

        
        journalist =c.scrap("'author-name':'(.*?)'",page)

        if not journalist: journalist = 'NA'

        # current date and time 00
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
        

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":foot_fall,
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content
                    } 

#         cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   

logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')